# Project: Data Cleaning Challenge

In [1]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import unicodedata


# display outputs inline
%matplotlib inline

# to see all columns and rows to using the display max column parameter
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# set the display colwidth for seeing the entire text contents
pd.set_option('display.max_colwidth', -1)

C:\Users\USER\AppData\Local\Temp\ipykernel_41772\2121266886.py:17: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
# Importing data
fifa = pd.read_csv('fifa21 raw data v2.csv')

C:\Users\USER\AppData\Local\Temp\ipykernel_41772\3160915589.py:2: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa = pd.read_csv('fifa21 raw data v2.csv')


In [3]:
# make a copy of the data so as not to tamper with the original copy
fifa_clean = fifa.copy()

# Data Inspection

In [4]:
# view a sample of the data
fifa_clean.sample(5)

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,Contract,Positions,Height,Weight,Preferred Foot,BOV,Best Position,Joined,Loan Date End,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
13674,213678,K. O'Hara,Kieran O'Hara,https://cdn.sofifa.com/players/213/678/21_60.png,http://sofifa.com/player/213678/kieran-ohara/210006/,Republic of Ireland,24,62,70,\n\n\n\nBurton Albion,2020 ~ 2022,GK,191cm,78kg,Right,62,GK,"Sep 11, 2020",NaN,€775K,€1K,€683K,80,12,14,15,26,13,88,16,13,11,29,19,255,49,41,50,58,57,207,43,61,41,51,11,112,25,25,13,36,13,44,48,15,14,19,303,63,59,57,60,64,1093,348,2 ★,1★,Medium,Medium,1 ★,63,59,57,64,45,60,4
11878,254198,A. Campos,Anyelo Campos,https://cdn.sofifa.com/players/254/198/21_60.png,http://sofifa.com/player/254198/anyelo-campos/210006/,Peru,27,64,67,\n\n\n\nMelgar FBC,2019 ~ 2020,GK,182cm,82kg,Right,64,GK,"Jan 1, 2019",NaN,€575K,€500,€890K,69,11,15,11,22,10,94,13,14,13,33,21,222,33,40,48,73,28,207,46,54,28,66,13,133,23,26,18,46,20,41,39,14,11,14,307,56,63,61,64,63,1071,344,2 ★,1★,Medium,Medium,1 ★,56,63,61,63,37,64,2
17666,245858,H. Seaden,Harry Seaden,https://cdn.sofifa.com/players/245/858/21_60.png,http://sofifa.com/player/245858/harry-seaden/210006/,England,19,55,70,\n\n\n\nSouthend United,2019 ~ 2021,GK,185cm,85kg,Right,55,GK,"Jul 1, 2019",NaN,€250K,€650,€333K,63,14,5,10,29,5,83,12,13,12,26,20,166,25,22,30,45,44,167,41,64,21,32,9,99,26,11,10,34,18,36,35,13,11,11,276,56,55,55,54,56,889,299,2 ★,1★,Medium,Medium,1 ★,56,55,55,56,23,54,NaN
7118,247257,Ibañez,Roger Ibañez Da Silva,https://cdn.sofifa.com/players/247/257/21_60.png,http://sofifa.com/player/247257/roger-ibanez-da-silva/210006/,Brazil,21,68,78,\n\n\n\nRoma,2019 ~ 2023,CB,186cm,74kg,Right,70,CB,"Jan 29, 2019",NaN,€2.5M,€6K,€3M,220,34,27,70,60,29,195,35,32,35,36,57,284,63,53,50,60,58,261,45,64,60,67,25,245,65,68,32,34,46,51,216,65,74,77,47,12,7,10,6,12,1468,314,2 ★,2★,Low,High,1 ★,58,32,43,46,70,65,43
3436,192129,K. Nordfeldt,Kristoffer Nordfeldt,https://cdn.sofifa.com/players/192/129/21_60.png,http://sofifa.com/player/192129/kristoffer-nordfeldt/210006/,Sweden,31,72,72,\n\n\n\nGençlerbirliği SK,2020 ~ 2021,GK,188cm,87kg,Right,72,GK,"Jan 16, 2020",NaN,€1.2M,€10K,€4M,89,11,13,13,41,11,95,14,20,11,26,24,277,52,49,58,72,46,254,52,83,33,74,12,109,24,23,15,34,13,28,35,7,13,15,354,71,66,69,74,74,1213,404,3 ★,1★,Medium,Medium,2 ★,71,66,69,74,50,74,8


#### Observations
1. Name of clubs are messy, they contain irrelevant characters
2. The Contract column values dont follow the same format
3. The Height column values arent well formatted so would likely give rise to wrong datatype, this is observed in the weight column too.
4. The weight column is graduated in the wrong units, it ought to be in lbs
5. Loan end date looks to contain lots of NAN values.
6. The Value, Wage and Release Clause columns arent well formatted, the currency type should be in dollars and some conversions should be made.
7. The W/F, SM and IR columns are not well formatted and would likely give rise to the wrong datatype
8. Some Column Names arent descriptive enough.
9. There are inconsistencies in some values, like in Hits column where some data have a K in them.

In [5]:
# Checking for duplicates
fifa_clean[fifa_clean.duplicated()]

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,Contract,Positions,Height,Weight,Preferred Foot,BOV,Best Position,Joined,Loan Date End,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits


#### No duplicate data in the dataset


In [6]:
# Identifying Null Values
null_counts = fifa_clean.isnull().sum()  # Count the number of null values in each column
null_cols = null_counts[null_counts > 0].index.tolist()  # Get the column names with null values

# Create a new dataframe with only the columns with null values and their corresponding null counts
null_fifa = pd.DataFrame({'Columns': null_cols, 'Null Count': null_counts[null_cols].tolist()})

null_fifa

,Columns,Null Count
0,Loan Date End,17966
1,Hits,2595


#### Observations
1. Only two columns contain Null Values, the Loan Date End and the Hits column, the Loan Date End Nulls seem to be significant.

In [7]:
# checking the datatypes and other information for each columns for each column
fifa_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                18979 non-null  int64 
 1   Name              18979 non-null  object
 2   LongName          18979 non-null  object
 3   photoUrl          18979 non-null  object
 4   playerUrl         18979 non-null  object
 5   Nationality       18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Club              18979 non-null  object
 10  Contract          18979 non-null  object
 11  Positions         18979 non-null  object
 12  Height            18979 non-null  object
 13  Weight            18979 non-null  object
 14  Preferred Foot    18979 non-null  object
 15  BOV               18979 non-null  int64 
 16  Best Position     18979 non-null  object
 17  Joined      

#### Observation
1. There are some inappropriate data type assigned ro some columns

In [8]:
# view summary statistics of the fifa data
fifa_clean.describe()

,ID,Age,↓OVA,POT,BOV,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,PAC,SHO,PAS,DRI,DEF,PHY
count,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.00000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000
mean,226403.384794,25.194109,65.718636,71.136414,66.751726,248.938142,49.688392,45.842405,51.942726,58.768112,42.696507,256.479214,55.587491,47.246957,42.391011,52.725381,58.528373,317.718689,64.357553,64.412930,63.366721,61.655619,63.925865,296.605195,57.801676,64.577375,62.653986,64.786922,46.785236,254.199747,55.591285,46.394489,50.298962,53.857105,48.057906,58.07366,139.909268,46.559197,47.705622,45.644449,81.436693,16.416302,16.207124,16.076453,16.217187,16.519627,1595.286949,355.702197,67.453975,53.457031,57.681016,62.875020,49.866221,64.368934
std,27141.054157,4.710520,6.968999,6.114635,6.747193,74.299428,18.131153,19.567081,17.294409,14.519106,17.646937,78.650601,18.761314,18.207790,17.227947,15.178151,16.565892,55.879046,14.889981,14.638743,14.596277,9.072114,14.062285,50.732390,13.308747,11.844870,15.804223,12.488672,19.300534,64.595613,17.140414,20.698078,19.428701,13.708574,15.655999,12.07911,61.219472,20.139324,21.367690,20.922087,84.665579,17.554194,16.816305,16.491103,17.002239,17.854079,269.874789,40.761117,10.677859,13.827425,10.081857,9.927415,16.443213,9.601883
min,41.000000,16.000000,47.000000,47.000000,48.000000,42.000000,6.000000,3.000000,5.000000,7.000000,3.000000,40.000000,5.000000,4.000000,5.000000,5.000000,5.000000,122.000000,13.000000,12.000000,14.000000,24.000000,12.000000,122.000000,18.000000,15.000000,12.000000,16.000000,4.000000,50.000000,9.000000,3.000000,2.000000,9.000000,6.000000,12.00000,20.000000,3.000000,5.000000,4.000000,10.000000,2.000000,2.000000,2.000000,2.000000,2.000000,747.000000,232.000000,25.000000,16.000000,25.000000,25.000000,12.000000,28.000000
25%,210135.000000,21.000000,61.000000,67.000000,62.000000,222.000000,38.000000,30.000000,44.000000,54.000000,30.000000,222.000000,49.000000,35.000000,31.000000,43.000000,54.000000,289.000000,57.000000,57.000000,55.000000,56.000000,56.000000,264.000000,48.000000,58.000000,55.000000,57.000000,32.000000,227.500000,44.000000,25.000000,40.000000,45.000000,39.000000,50.00000,83.000000,29.000000,27.000000,25.000000,48.000000,8.000000,8.000000,8.000000,8.000000,8.000000,1452.000000,327.000000,61.000000,44.000000,51.000000,57.000000,35.000000,58.000000
50%,232418.000000,25.000000,66.000000,71.000000,67.000000,263.000000,54.000000,49.000000,55.000000,62.000000,44.000000,269.000000,61.000000,49.000000,41.000000,56.000000,63.000000,327.000000,67.000000,67.000000,66.000000,62.000000,66.000000,302.000000,59.000000,65.000000,66.000000,66.000000,51.000000,263.000000,58.000000,53.000000,55.000000,55.000000,49.000000,59.00000,159.000000,52.000000,55.000000,52.000000,53.000000,11.000000,11.000000,11.000000,11.000000,11.000000,1627.000000,356.000000,68.000000,56.000000,58.000000,64.000000,53.000000,65.000000
75%,246922.500000,29.000000,70.000000,75.000000,71.000000,297.

# Data Cleaning

### Handling inconsistencies in the data values

#### Rename Columns

In [9]:
fifa_clean.columns

Index(['ID', 'Name', 'LongName', 'photoUrl', 'playerUrl', 'Nationality', 'Age',
       '↓OVA', 'POT', 'Club', 'Contract', 'Positions', 'Height', 'Weight',
       'Preferred Foot', 'BOV', 'Best Position', 'Joined', 'Loan Date End',
       'Value', 'Wage', 'Release Clause', 'Attacking', 'Crossing', 'Finishing',
       'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill', 'Dribbling',
       'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Movement',
       'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
       'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots',
       'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle',
       'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats',
       'Base Stats', 'W/F', 'SM', 'A/W', 'D/W', 'IR', 'PAC', 'SHO', 'PAS',
       'DRI', 'DEF', 

In [10]:
# Give columns more descriptive names for easy understanding, Abbreviations can be ambiguous and can lead to misunderstandings when analyzing the data
fifa_clean.rename(columns = {'LongName':'Full Name','Age':'Age(2021)', '↓OVA':'Overall Rating', 'POT':'Potential Rating',
                             'Height':'Height(ft)', 'Weight':'Weight(lbs)', 'BOV':'Best Overall Rating',
                            'Value':'Value(€)', 'Wage':'Wage(€)', 'ReleaseClause':'Release Clause(€)',
                            'W/F':'Weaker Foot(★)', 'SM':'Skils(★)', 'A/W':'Attacking Contribution',
                             'D/W':'Defensive Contribution', 'IR':'Injury Resistance(★)', 'PAC':'Pace', 'SHO':'Shot', 
                             'PAS':'Pass Accuracy', 'DRI':'Dribbling Ability',
                            'DEF':'Defensive Ability', 'PHY': 'Physicality', 'Hits': 'Profile Views'}, inplace = True)

fifa_clean.columns

Index(['ID', 'Name', 'Full Name', 'photoUrl', 'playerUrl', 'Nationality',
       'Age(2021)', 'Overall Rating', 'Potential Rating', 'Club', 'Contract',
       'Positions', 'Height(ft)', 'Weight(lbs)', 'Preferred Foot',
       'Best Overall Rating', 'Best Position', 'Joined', 'Loan Date End',
       'Value(€)', 'Wage(€)', 'Release Clause', 'Attacking', 'Crossing',
       'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill',
       'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control',
       'Movement', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions',
       'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength',
       'Long Shots', 'Mentality', 'Aggression', 'Interceptions', 'Positioning',
       'Vision', 'Penalties', 'Composure', 'Defending', 'Marking',
       'Standing Tackle', 'Sliding Tackle', 'Goalkeeping', 'GK Diving',
       'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes',
       'Total Stats', 'Base Stats', 'Weaker

#### Datatype Conversion

In [11]:
fifa_clean['ID'] = fifa_clean['ID'].astype(str)
fifa_clean['Preferred Foot'] = fifa_clean['Preferred Foot'].astype('category')
fifa_clean['Joined'] = pd.to_datetime(fifa_clean['Joined'])
fifa_clean['Loan Date End'] = pd.to_datetime(fifa_clean['Loan Date End'])
fifa_clean['Weaker Foot(★)'] = fifa_clean['Weaker Foot(★)'].astype('category')
fifa_clean['Skils(★)'] = fifa_clean['Skils(★)'].astype('category')
fifa_clean['Attacking Contribution'] = fifa_clean['Attacking Contribution'].astype('category')
fifa_clean['Defensive Contribution'] = fifa_clean['Defensive Contribution'].astype('category')
fifa_clean['Injury Resistance(★)'] = fifa_clean['Injury Resistance(★)'].astype('category')

#### Cleaning the Profile Views Column

This column has 2595 missing  values, values of over 1000 are represented with a decimal and a k, handling these issue would make the column cleaner, To fill the missing values, the mode would be used as the data is skewed

In [12]:
# looking at the Profile Views column to observe inconsistencies
fifa_clean['Profile Views'].loc[6:8]

6    246 
7    120 
8    1.6K
Name: Profile Views, dtype: object

In [13]:
# Replace 'K' and 'M' with their corresponding numeric values in the 'Profile Views' column
fifa_clean['Profile Views'] = fifa_clean['Profile Views'].str.replace('K', 'e3').str.replace('M', 'e6')

# Convert the 'Profile Views' column to numeric type
fifa_clean['Profile Views'] = pd.to_numeric(fifa_clean['Profile Views'])

# Filter rows where the 'Profile Views' column contains 'K'
k_values = fifa_clean[fifa_clean['Profile Views'].astype(str).str.contains('e3')]

fifa_clean['Profile Views'].loc[6:8]

6    246.0 
7    120.0 
8    1600.0
Name: Profile Views, dtype: float64

In [14]:
# check the skewness of the hits values to determine what measure of center is best to fill null values
fifa_clean['Profile Views'].skew()

31.04253072771258

In [15]:
# get the mode to fill in na with
mode_hits = fifa_clean['Profile Views'].mode()
mode_hits

0    1.0
Name: Profile Views, dtype: float64

In [16]:
# fill in missing values in Profile Views column
fifa_clean['Profile Views'].fillna(1.0, inplace=True)

# change profile views to int data type
fifa_clean['Profile Views'] = fifa_clean['Profile Views'].astype(int)

# Check to confirm there are no Null values
fifa_clean[fifa_clean['Profile Views'].isna()]

,ID,Name,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Contract,Positions,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Loan Date End,Value(€),Wage(€),Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views


####  Cleaning the Contract column

The Contract Column is having many discrepancies in value format, it also has some relationship with the Joined and loan date end column, from the contract column, a contract type column would be derived, this column tells us if the players contract is a permanent deal, a loan deal or a free agent(not under contract with any football club and is therefore free to sign with any club of their choosing). After which a contract start and contract end column is derived from the contract column, joined column and loan date column, these new columns tells when a player contract started, for a free agent this would be non applicable (N/A) as he has no contract while the contract end tells when the contracts would be terminated, for free agents the contract end would be N/A (not applicable) as a player without a contract cant have end date

In [17]:
fifa_clean['Contract'].loc[397:405]

397    Free                
398    2019 ~ 2023         
399    2017 ~ 2023         
400    2019 ~ 2024         
401    2014 ~ 2026         
402    2018 ~ 2023         
403    2019 ~ 2023         
404    2019 ~ 2024         
405    Jun 30, 2021 On Loan
Name: Contract, dtype: object

In [18]:
fifa_clean['Contract Type'] = fifa_clean['Contract'].apply(lambda x: 'Free' if x == 'Free' 
                                         else 'On Loan' if 'On Loan' in x 
                                         else 'Permanent' if '~' in x 
                                         else np.nan)


In [19]:
# define a function to parse the values in the 'Joined' and 'Loan Date End' columns
def parse_dates(row):
    if row['Contract'] == 'Free':
        start = np.nan
        end = np.nan
    elif 'On Loan' in row['Contract']:
        start = pd.to_datetime(row['Joined']).year
        end = row['Contract'].split(' ')[2]
    else:
        start = int(row['Contract'][:4])
        end = int(row['Contract'][-4:])
    return pd.Series({'Contract Start': start, 'Contract End': end})

# apply the function to create the new columns
fifa_clean[['Contract Start', 'Contract End']] = fifa_clean.apply(parse_dates, axis=1)


# convert float columns to object datatype and remove the unwanted characters
fifa_clean['Contract Start'] = fifa_clean['Contract Start'].astype(str)
fifa_clean['Contract End'] = fifa_clean['Contract End'].astype(str)

# Define a function to remove the '.i0' part from a string
def remove_dot_i(s):
    return s.split('.')[0]

# Apply the function to the fifa_clean['Contract Start']  and fifa_clean['Contract end'] column using .map()
fifa_clean['Contract Start'] = fifa_clean['Contract Start'].map(remove_dot_i)
fifa_clean['Contract End'] = fifa_clean['Contract End'].map(remove_dot_i)

# Assigning the contract start and contract end columns into categories
fifa_clean['Contract Start'] = fifa_clean['Contract Start'].astype('category')
fifa_clean['Contract End'] = fifa_clean['Contract End'].astype('category')

# view samples of the result
fifa_clean[['Contract Start', 'Contract End']].loc[397:405]

,Contract Start,Contract End
397,nan,nan
398,2019,2023
399,2017,2023
400,2019,2024
401,2014,2026
402,2018,2023
403,2019,2023
404,2019,2024
405,2017,2021


#### Cleaning the values in the Club, Name and Full Name columns

The club column values have messy and irrelevant characters in them, this would be cleaned. The Name, Full Name and Club colums have some diacritic characters whic could arise due to different languages and would affect alphabetic ordering during visualizations

In [20]:
# take a look at the column to see the format of naming
fifa_clean['Club'].sample(3)

6137     \n\n\n\nCD Nacional           
17766    \n\n\n\nIFK Göteborg          
8486     \n\n\n\nClub Atlético Talleres
Name: Club, dtype: object

In [21]:
# Replacing the '\n' with an empty string
fifa_clean['Club'] = fifa_clean['Club'].str.replace('\n', '')
fifa_clean['Club'].sample(3)

15566    SV Waldhof Mannheim
2315     Peñarol            
7328     Real Zaragoza      
Name: Club, dtype: object

In [22]:
# Examples of diacritic names
fifa_clean[['Name','Full Name','Club']].loc[110]

Name         N. Süle          
Full Name    Niklas Süle      
Club         FC Bayern München
Name: 110, dtype: object

In [23]:
# Define regular expression pattern to match special characters in Name, LongName and Club columns
def remove_diacritics(text):
    """
    This function removes diacritics (accent marks) from text
    """
    return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')

# Apply remove_diacritics function to Name, LongName, and Club columns
fifa_clean['Name'] = fifa_clean['Name'].apply(remove_diacritics)
fifa_clean['Full Name'] = fifa_clean['Full Name'].apply(remove_diacritics)
fifa_clean['Club'] = fifa_clean['Club'].apply(remove_diacritics)

# Check if Diacritic Names were removed
fifa_clean[['Name','Full Name','Club']].loc[110]

Name         N. Sule          
Full Name    Niklas Sule      
Club         FC Bayern Munchen
Name: 110, dtype: object

### 

####  Cleaning the Height column
The height column has values with contrasting units, some are in cm and others in feet, to clean it the units are removed from the values and then they are all converted to ftin in decimals, a descriptive column name would ensure the values are easily understood. The ftin unit is a more appropriate unit here because the weight column is tobe formatted in lbs, with this if need be for creating a BMI column it wont need any more formatting

In [24]:
fifa_clean['Height(ft)'].sample(6)

4381     184cm
7838     168cm
15725    178cm
18771    168cm
7815     176cm
11853    174cm
Name: Height(ft), dtype: object

In [25]:
# define a function that converts cm to feets and inches, This is my preferred unit as the weight is in lbs, this makes calculations that demands precision like the BMI easy

def convert_cm_to_feet(cm):
    cm = float(cm.replace('cm', '')) # remove 'cm' units and convert to float
    feet = int(cm / 30.48) # calculate number of feet
    inches = round((cm / 2.54) % 12) # calculate number of inches and round to nearest integer
    return f"{feet}'{inches}\"" # return result in feet and inches format

# apply the conversion function only to values that are in cm
fifa_clean['Height(ft)'] = fifa_clean['Height(ft)'].apply(lambda x: convert_cm_to_feet(x) if 'cm' in x else x)

# view the result
fifa_clean['Height(ft)'].sample(5)

856      5'11"
17403    5'10"
9743     6'3" 
10851    6'4" 
5266     6'1" 
Name: Height(ft), dtype: object

In [26]:
# define a function to convert feet and inches format to decimal format
def convert_feet_to_dec(feet):
    feet_list = feet.split("'")
    ft = float(feet_list[0])
    if len(feet_list) > 1:
        inches = float(feet_list[1].replace('"', ''))
    else:
        inches = 0
    dec = ft + inches / 12
    return round(dec, 2) # round to 1 decimal place


# apply the conversion function to the 'Height' column
fifa_clean['Height(ft)'] = fifa_clean['Height(ft)'].apply(convert_feet_to_dec)

# view result
fifa_clean['Height(ft)'].sample(6)

16029    6.00
12982    5.58
893      6.17
2697     5.58
14289    6.33
15574    5.75
Name: Height(ft), dtype: float64

#### Cleaning the Weight Column
The weight column has values with contrasting units, some are in lbs and others in kg, to clean it the units are removed from the values and then they are all converted to lbs, a descriptive column name would ensure the values are easily understood. The lbs unit is a more appropriate unit here because the height column is formatted in ftin, with this if need be for creating a BMI column it wont need any more formatting

In [27]:
fifa_clean['Weight(lbs)'].sample(6)

3192     84kg
18001    65kg
17783    64kg
14631    74kg
2978     77kg
7171     88kg
Name: Weight(lbs), dtype: object

In [28]:
# define a function that converts kg to lbs
def convert_kg_to_lbs(kg):
    kg = float(kg.replace('kg', '')) # remove 'kg' units and convert to float
    lbs = int(kg * 2.20462) # calculate kg
    return f"{lbs}lbs" #returns result in lbs format

# apply the conversion function only to values that are in kg
fifa_clean['Weight(lbs)'] = fifa_clean['Weight(lbs)'].apply(lambda x: convert_kg_to_lbs(x) if 'kg' in x else x)

# remove the lbs units from the vale
fifa_clean['Weight(lbs)'] = fifa_clean['Weight(lbs)'].str.replace('lbs', '')

# convert to float datatype
fifa_clean['Weight(lbs)'] = fifa_clean['Weight(lbs)'].astype(float)

# view result of code
fifa_clean['Weight(lbs)'].sample(3)

15024    169.0
11084    163.0
6728     145.0
Name: Weight(lbs), dtype: float64

#### Cleaning the Value, Wage and Release Clause columns
The Value, Wage and Release columns have similar mess issues, have currency symbols in them and are formatted in thousand and millions decimal points and have K and M attached to them. These symbols would be dropped and the thousands and millions format would be made uniform. The choice of of the euro(€) currency is because most transfer deals are negotiated in euros

In [29]:
fifa_clean[['Value(€)','Wage(€)', 'Release Clause']].head()

,Value(€),Wage(€),Release Clause
0,€103.5M,€560K,€138.4M
1,€63M,€220K,€75.9M
2,€120M,€125K,€159.4M
3,€129M,€370K,€161M
4,€132M,€270K,€166.5M


In [30]:
# Define a function to convert euro and k/m suffixes to USD and millions
def convert_value(val):
    if val.startswith('€'):
        val = val.replace('€', '')
        if val.endswith('K'):
            val = float(val.replace('K', '')) * 1000
        elif val.endswith('M'):
            val = float(val.replace('M', '')) * 1000000
    
    return pd.to_numeric(val)

# Apply the conversion function to the DataFrame
fifa_clean[['Value(€)', 'Wage(€)', 'Release Clause']] = fifa_clean[['Value(€)', 'Wage(€)', 'Release Clause']].applymap(convert_value)

# Set the display format of floats to include a comma delimiter
pd.options.display.float_format = '{:,.2f}'.format

fifa_clean.rename(columns={'Release Clause': 'Release Clause(€)'}, inplace=True)

# quick glance at the resulting data
fifa_clean[['Value(€)','Wage(€)', 'Release Clause(€)']].sample(5)

,Value(€),Wage(€),Release Clause(€)
790,"20,000,000.00","42,000.00","24,700,000.00"
1952,"7,500,000.00","45,000.00","14,200,000.00"
4008,"1,300,000.00",500.00,"3,700,000.00"
7896,"800,000.00","4,000.00","780,000.00"
717,"22,500,000.00","56,000.00","27,600,000.00"


#### Cleaning columns with star ratings (Weaker Foot, Skills, Injury Resistance)
The columns with values as star ratings have symbols to represent the number of stars received for each category by a player, these star symbols would be dropped

In [31]:
fifa_clean[['Weaker Foot(★)','Skils(★)','Injury Resistance(★)']].head()

,Weaker Foot(★),Skils(★),Injury Resistance(★)
0,4 ★,4★,5 ★
1,4 ★,5★,5 ★
2,3 ★,1★,3 ★
3,5 ★,4★,4 ★
4,5 ★,5★,5 ★


In [32]:
fifa_clean['Weaker Foot(★)'] = fifa_clean['Weaker Foot(★)'].str.replace('\u2605', '')
fifa_clean['Skils(★)'] = fifa_clean['Skils(★)'].str.replace('\u2605', '')
fifa_clean['Injury Resistance(★)'] = fifa_clean['Injury Resistance(★)'].str.replace('\u2605', '')

# quick glance at the resulting data
fifa_clean[['Weaker Foot(★)','Skils(★)','Injury Resistance(★)']].sample(5)

,Weaker Foot(★),Skils(★),Injury Resistance(★)
18873,2,2,1
2437,3,2,1
2260,3,3,1
11217,3,2,1
15258,3,1,1


#### Current ages of players from the Age column 
The age column houses ages of players in 2021 when this data was collated to have an updated age value for the year of 2023, a new column would be derived from the previous age column

In [33]:
# add 2 to their previous age to get their current age
fifa_clean['Age(2023)'] = fifa_clean['Age(2021)'] + 2

# a quick glance of samples of the data
fifa_clean[['Age(2021)', 'Age(2023)']].sample(5)

,Age(2021),Age(2023)
12552,23,25
1066,25,27
8568,24,26
16331,24,26
5810,25,27


#### Drop unnecessary columns
The contract and loan date end columns have been formatted already to give newer more imformative and cleaner columns(Contract Type, Contract Start and Contract End) so should be drop, the positions column looks messy and a more unique column(Best Positions) is available to tell us what position the player plays at

In [34]:
# drop the contract, positions and loan date end columns
fifa_clean.drop(['Contract', 'Positions', 'Name', 'Loan Date End'], axis=1, inplace=True)
fifa_clean.columns

Index(['ID', 'Full Name', 'photoUrl', 'playerUrl', 'Nationality', 'Age(2021)',
       'Overall Rating', 'Potential Rating', 'Club', 'Height(ft)',
       'Weight(lbs)', 'Preferred Foot', 'Best Overall Rating', 'Best Position',
       'Joined', 'Value(€)', 'Wage(€)', 'Release Clause(€)', 'Attacking',
       'Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys',
       'Skill', 'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing',
       'Ball Control', 'Movement', 'Acceleration', 'Sprint Speed', 'Agility',
       'Reactions', 'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina',
       'Strength', 'Long Shots', 'Mentality', 'Aggression', 'Interceptions',
       'Positioning', 'Vision', 'Penalties', 'Composure', 'Defending',
       'Marking', 'Standing Tackle', 'Sliding Tackle', 'Goalkeeping',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'Total Stats', 'Base Stats', 'Weaker Foot(★)',
       'Skils(★)', 'Attacking Contribution'

#### The total stats column is an aggregate of the Attacking, Skill, Movement, Power, Mentality, Defending and Goal Keeping Column, each of these columns are also aggregates of other columns which are broken down below:
1.  Attacking Column = Crossing + Finishing + HEading Accuracy + Short Passing + Volleys
2. Skill Column = Dribbling + Curve + FK Accuracy + Long Passing + Ball Control 
3. Movement Column = Acceleration + Sprint Speed + Agility + Reactions + Balance 
4. Power Column = Shot Power + Jumping + Stamina + Strength + Long Shots
5. Mentality Column = Aggression + Interceptions + Positioning + Vision + Penalties
6. Defending Column = Marking + Standing Tackle + Sliding Tackle
7. Goalkeeping = GK Diving + GK Handling + GK Kicking + GK Positioning + GK Reflexes
#### This leaves out the composure column, a quick visit to the sofifa.com site shows this is a little oversight and the composure column was supposed to be added to the Mentality Column and the total affects the total stats.

Other attributes column would be accessed to be sure the calculations done were correct and then the composure column values would be inputted where they ought to bee used for calculations

##### Confirm Attacking column Calculations

In [35]:
# Create a new column "Total Attacking" to store the sum of the attacking attributes
fifa_clean["Total Attacking"] = fifa_clean[["Crossing", "Finishing", "Heading Accuracy", "Short Passing", "Volleys"]].sum(axis=1)

# Check if the "Total Attacking" matches the "Attacking" column for each row
fifa_clean["Attacking Check"] = fifa_clean["Attacking"] == fifa_clean["Total Attacking"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean['Attacking Check'] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Attacking,Attacking Check


##### drop columns used in comparison

In [36]:
# Drop the "Total Attacking" and "Attacking Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Attacking", "Attacking Check"])

# a quick glance to confirm thy were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
6347,186359,Sergio Aquino,https://cdn.sofifa.com/players/186/359/21_60.png,http://sofifa.com/player/186359/sergio-aquino/210006/,Paraguay,40,69,69,Club Libertad,5.75,158.00,Right,69,LW,2006-01-01,0.00,500.00,0.00,269,69,63,32,69,36,281,68,59,40,45,69,334,79,71,73,68,43,301,51,32,86,63,69,278,50,39,69,74,46,55,141,57,44,40,59,8,14,14,13,10,1663,377,2,3,Medium,Medium,1,75,60,64,68,45,65,3,Permanent,2006,2023,42


##### Confirm Skill column Calculations

In [37]:
# Create a new column "Total Skill" to store the sum of the skill attributes
fifa_clean["Total Skill"] = fifa_clean[['Dribbling','Curve', 'FK Accuracy', 'Long Passing', 'Ball Control']].sum(axis=1)

# Check if the "Total Skill" matches the "Skill" column for each row
fifa_clean["Skill Check"] = fifa_clean["Skill"] == fifa_clean["Total Skill"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean['Skill Check'] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Skill,Skill Check


##### drop columns used in comparison

In [38]:
# Drop the "Total Skill" and "Skill Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Skill", "Skill Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
10437,253882,Cesar Urpin,https://cdn.sofifa.com/players/253/882/21_60.png,http://sofifa.com/player/253882/cesar-urpin/210006/,Venezuela,25,65,67,Aragua FC,6.00,156.00,Right,66,CB,2017-01-09,"825,000.00",500.00,"1,300,000.00",230,48,39,60,51,32,239,55,39,41,45,59,338,76,62,63,56,81,281,31,79,72,60,39,261,63,61,50,45,42,52,206,65,69,72,45,12,8,8,6,11,1600,342,2,2,Medium,Medium,1,68,38,47,58,66,65,2,Permanent,2017,2024,27


##### Confirm Movement column Calculations

In [39]:
# Create a new column "Total Movement" to store the sum of the Movement attributes
fifa_clean["Total Movement"] = fifa_clean[['Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance']].sum(axis=1)

# Check if the "Total Movement" matches the "Movement" column for each row
fifa_clean["Movement Check"] = fifa_clean["Movement"] == fifa_clean["Total Movement"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean['Movement Check'] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Movement,Movement Check


##### drop columns used in comparison

In [40]:
# Drop the "Total Movement" and "Movement Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Movement", "Movement Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
1174,197083,Daniel Caligiuri,https://cdn.sofifa.com/players/197/083/21_60.png,http://sofifa.com/player/197083/daniel-caligiuri/210006/,Italy,32,77,77,FC Augsburg,6.00,176.00,Right,77,RM,2020-07-01,"8,500,000.00","29,000.00","12,600,000.00",357,80,71,63,76,67,377,75,78,78,70,76,355,68,68,74,78,67,382,76,77,82,73,74,376,72,74,72,75,83,74,212,71,73,68,55,12,10,13,6,14,2114,438,4,3,Medium,Medium,2,68,73,76,75,71,75,19,Permanent,2020,2023,34


##### Confirm Power column Calculations

In [41]:
# Create a new column "Total Power" to store the sum of the Power attributes
fifa_clean["Total Power"] = fifa_clean[['Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots']].sum(axis=1)

# Check if the "Total Power" matches the "Power" column for each row
fifa_clean["Power Check"] = fifa_clean["Power"] == fifa_clean["Total Power"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean["Power Check"] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Power,Power Check


##### drop columns used in comparison

In [42]:
# Drop the "Total Power" and "Power Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Power", "Power Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
3724,199897,Nicolas Hofler,https://cdn.sofifa.com/players/199/897/21_60.png,http://sofifa.com/player/199897/nicolas-hofler/210006/,Germany,30,72,72,SC Freiburg,5.92,178.00,Right,72,CDM,2010-07-01,"2,000,000.00","16,000.00","4,700,000.00",270,54,41,61,74,40,303,64,53,50,68,68,319,54,63,66,71,65,334,58,84,73,70,49,303,66,75,49,63,50,76,211,72,70,69,51,6,13,13,10,9,1791,379,3,2,Medium,Medium,1,59,47,65,66,71,71,4,Permanent,2010,2021,32


##### Update Mentality column by adding Composure values to its attributes

In [43]:
# suming  up the  'Aggression', 'Interceptions', 'Positioning', 'Vision','Penalties'and add the 'Composure' column to make the Mentality column
fifa_clean['Mentality'] = fifa_clean[[ 'Aggression', 'Interceptions', 'Positioning', 'Vision','Penalties', 'Composure']].sum(axis=1)

# a quick glance to confirm they were dropped
fifa_clean[['Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure']].sample(4)

,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure
2173,360,77,74,38,60,44,67
15899,286,53,55,48,46,36,48
12718,303,59,58,47,51,33,55
18394,226,47,54,21,27,37,40


##### Confirm Defending column Calculations

In [44]:
# Create a new column "Total Defending" to store the sum of the defending attributes
fifa_clean["Total Defending"] = fifa_clean[['Marking', 'Standing Tackle','Sliding Tackle']].sum(axis=1)

# Check if the "Total Defending" matches the "defending" column for each row
fifa_clean["Defending Check"] = fifa_clean["Defending"] == fifa_clean["Total Defending"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean["Defending Check"] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Defending,Defending Check


##### drop columns used in comparison

In [45]:
# Drop the "Total Defending" and "Defending Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Defending", "Defending Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
6647,213839,Baptiste Guillaume,https://cdn.sofifa.com/players/213/839/21_60.png,http://sofifa.com/player/213839/baptiste-guillaume/210006/,Belgium,25,68,73,Valenciennes FC,6.25,189.00,Right,70,ST,2020-07-09,"1,800,000.00","3,000.00","2,200,000.00",319,55,66,73,67,58,255,69,36,29,51,70,301,63,49,68,65,56,335,71,69,61,76,58,313,49,19,71,52,57,65,81,41,18,22,50,13,10,8,12,7,1589,342,4,3,Medium,Medium,1,55,65,56,68,31,67,2,Permanent,2020,2023,27


##### Confirm Goalkeeping column Calculations

In [46]:
# Create a new column "Total GK" to store the sum of the Goalkeeping attributes
fifa_clean["Total GK"] = fifa_clean[['GK Diving', 'GK Handling','GK Kicking', 'GK Positioning', 'GK Reflexes']].sum(axis=1)

# Check if the "Total Defending" matches the "defending" column for each row
fifa_clean["GK Check"] = fifa_clean["Goalkeeping"] == fifa_clean["Total GK"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean["GK Check"] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total GK,GK Check


##### drop columns used in comparison

In [47]:
# Drop the "Total GK" and "GK Check" columns
fifa_clean = fifa_clean.drop(columns=["Total GK", "GK Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
6278,208111,Simon Sandberg,https://cdn.sofifa.com/players/208/111/21_60.png,http://sofifa.com/player/208111/simon-sandberg/210006/,Sweden,26,69,72,Hammarby IF,6.08,169.00,Right,69,RB,2018-01-01,"1,700,000.00","3,000.00","1,500,000.00",270,70,41,60,64,35,279,60,60,35,62,62,338,68,76,66,65,63,308,58,66,78,72,34,348,69,68,56,55,39,61,192,66,66,60,51,9,10,14,7,11,1725,376,3,2,Medium,Medium,1,72,43,61,62,65,73,4,Permanent,2018,2022,28


#### Total stats represent the final value of all the stats a character or object has, which includes their base stats, any bonuses or modifications gained through equipment or leveling up, and any other factors that affect their abilities. Therefore, adding the base stats to the total stats would result in double-counting the base stats. So base stats would be left out in this aggregation

##### Update Total Stats column by with updating values of its attributes

In [48]:
# suming  up the Attacking, Skill, Movement, Power, Mentality, Defending and Goal Keeping to make the Mentality column
fifa_clean['Total Stats'] = fifa_clean[[ 'Attacking', 'Skill', 'Movement', 'Power','Mentality', 'Defending']].sum(axis=1)

# a view of the resulting total stats column
fifa_clean[[ 'Total Stats','Attacking', 'Skill', 'Movement', 'Power','Mentality', 'Defending']].sample()

,Total Stats,Attacking,Skill,Movement,Power,Mentality,Defending
12685,646,60,64,185,187,123,27


#### Confirm Base Stats column Calculations
The Base stats refer to the six basic categories of skills used to rate a player's abilities in FIFA. These stats are typically used as a starting point for evaluating a player's performance, and they provide a basic overview of a player's abilities. The six columns would be added to compare if they sum up to the base stats

##### Confirm Base Stats column Calculations

In [49]:
# Create a new column "Total Base Stats" to store the sum of the Base Stats attributes
fifa_clean["Total Base Stats"] = fifa_clean[['Pace', 'Shot',
       'Pass Accuracy', 'Dribbling Ability', 'Defensive Ability',
       'Physicality']].sum(axis=1)

# Check if the "Total Defending" matches the "defending" column for each row
fifa_clean["Base Stats Check"] = fifa_clean["Base Stats"] == fifa_clean["Total Base Stats"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean["Base Stats Check"] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Base Stats,Base Stats Check


##### drop columns used in comparison

In [50]:
# Drop the "Total Base Stats" and "Base Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Base Stats", "Base Stats Check"])

# a view of a sample of the data
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
7482,250918,Said Bakari,https://cdn.sofifa.com/players/250/918/21_60.png,http://sofifa.com/player/250918/said-bakari/210006/,Comoros,25,67,70,RKC Waalwijk,5.83,147.00,Right,68,RM,2017-07-01,"1,200,000.00","2,000.00","1,300,000.00",302,63,56,59,67,57,289,71,58,45,48,67,379,84,81,77,63,74,307,63,60,68,68,48,370,71,65,62,59,52,61,173,54,60,59,51,12,13,10,9,7,1820,395,4,3,High,Medium,1,82,56,60,70,59,68,4,Permanent,2017,2023,27


#### Reindex Columns

Changing the order of columns can be helpful for organizing the data in a way that makes it easier to understand or analyze.

In [51]:
fifa_clean.insert(loc=7, column='Age(2023)', value=fifa_clean.pop('Age(2023)'))
fifa_clean.insert(loc=8, column='Contract Type', value=fifa_clean.pop('Contract Type'))
fifa_clean.insert(loc=9, column='Contract Start', value=fifa_clean.pop('Contract Start'))
fifa_clean.insert(loc=10, column='Contract End', value=fifa_clean.pop('Contract End'))
fifa_clean.insert(loc=66, column='Pace', value=fifa_clean.pop('Pace'))
fifa_clean.insert(loc=67, column='Shot', value=fifa_clean.pop('Shot'))
fifa_clean.insert(loc=68, column='Pass Accuracy', value=fifa_clean.pop('Pass Accuracy'))
fifa_clean.insert(loc=69, column='Dribbling Ability', value=fifa_clean.pop('Dribbling Ability'))
fifa_clean.insert(loc=70, column='Defensive Ability', value=fifa_clean.pop('Defensive Ability'))
fifa_clean.insert(loc=71, column='Physicality', value=fifa_clean.pop('Physicality'))


fifa_clean.sample(3)

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Age(2023),Contract Type,Contract Start,Contract End,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Profile Views
985,203106,Leonardo Bittencourt,https://cdn.sofifa.com/players/203/106/21_60.png,http://sofifa.com/player/203106/leonardo-bittencourt/210006/,Germany,26,77,28,Permanent,2020,2024,78,SV Werder Bremen,5.58,136.00,Right,77,RM,2020-07-01,"12,500,000.00","29,000.00","19,700,000.00",333,73,68,46,77,69,384,86,77,70,72,79,412,85,74,90,74,89,325,70,65,76,46,68,402,74,55,68,72,61,72,172,58,57,57,62,15,11,6,14,16,2028,421,4,79,68,74,84,56,60,3,Medium,Medium,2,30
16642,254462,Guido Chavez,https://cdn.sofifa.com/players/254/462/21_60.png,http://sofifa.com/player/254462/guido-chavez/210006/,Paraguay,21,58,23,Permanent,2016,2021,69,Sol de America,6.08,180.00,Right,60,ST,2016-01-01,"450,000.00",500.00,"517,000.00",284,58,60,57,48,61,282,60,62,45,57,58,277,62,67,48,50,50,328,62,66,75,70,55,339,65,42,55,54,64,59,134,39,49,46,50,10,10,13,8,9,1644,349,3,65,59,53,57,45,70,2,High,Low,1,1
893,154472,Jorge Molina Vidal,https://cdn.sofifa.com/players/154/472/21_60.png,http://sofifa.com/player/154472/jorge-molina-vidal/210005/,Spain,38,78,40,Permanent,2020,2022,78,Granada CF,6.17,159.00,Right,78,ST,2020-08-24,"3,700,000.00","24,000.00","7,800,000.00",377,49,84,83,80,81,330,73,62,59,57,79,226,33,39,33,85,36,337,80,46,43,89,79,440,77,41,87,74,80,81,96,45,32,19,55,6,14,8,13,14,1806,370,4,36,82,67,70,42,73,3,Medium,Medium,1,11


In [52]:
# save cleaned fifa data
fifa_clean.to_csv('Fifa_2021_cleaned.csv',index=False)

# Print a message to confirm the file has been saved
print('fifa_clean.csv saved successfully.')

fifa_clean.csv saved successfully.
